# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [5]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [8]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [9]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [10]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [11]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [12]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [13]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [14]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document

Topic: Artificial Intelligence

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of Artificial Intelligence in today's world
- Brief overview of the latest trends in AI

II. Latest Trends in Artificial Intelligence
- Deep Learning and Neural Networks
- Natural Language Processing
- Computer Vision
- AI in healthcare, finance, and other industries
- AI ethics and regulations

III. Key Players in Artificial Intelligence
- Google
- Amazon
- Microsoft
- IBM
- Te

- Display the results of your execution as markdown in the notebook.

In [16]:
from IPython.display import Markdown
Markdown(result)

# The Rise of Artificial Intelligence: Trends, Players, and Implications

Artificial Intelligence (AI) has permeated every aspect of our modern world, reshaping industries and redefining the way we interact with technology. By replicating human intelligence processes in machines, AI has significantly boosted efficiency, accuracy, and innovation across diverse sectors. The latest trends in AI encompass cutting-edge technologies like Deep Learning, Neural Networks, Natural Language Processing, and Computer Vision, which have paved the way for groundbreaking advancements in healthcare, finance, and beyond, fostering economic growth and societal advancement.

In the realm of AI, Deep Learning and Neural Networks stand at the forefront of research, empowering machines to learn from vast datasets and execute complex tasks with human-like precision. Natural Language Processing has revolutionized human-machine communication, giving rise to virtual assistants and language translation tools. Moreover, Computer Vision has amplified image recognition capabilities, propelling advancements in autonomous vehicles and surveillance systems. In critical domains such as healthcare and finance, AI applications have bolstered diagnosis accuracy, personalized treatment plans, and financial forecasting, significantly enhancing operational efficiency and outcomes.

When it comes to the key players in the AI landscape, leading tech giants including Google, Amazon, Microsoft, IBM, Tesla, and OpenAI have been instrumental in driving AI innovation and shaping the trajectory of technology. These industry frontrunners invest substantially in AI research and development, crafting cutting-edge solutions that tackle real-world challenges and opportunities head-on. Noteworthy initiatives like Google's DeepMind project, Amazon's Alexa virtual assistant, and IBM's Watson AI platform have left an indelible mark on industries and consumer experiences, showcasing the transformative power of AI on a global scale.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [17]:
topic = "When men will get Mars?"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on When men will get Mars?.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: When Will Men Get to Mars? Exploring the Latest Trends and Predictions

Introduction:
- Brief overview of space exploration history
- Introduction to the topic of when humans will reach Mars
- Mention of key players in the space exploration industry
- Brief explanation of the importance of reaching Mars

Key Points:
1. Latest Trends in Mars Exploration:
- Overview of recent Mars missions and discoveries
- Updates on current Mars missions by NASA, SpaceX, and other space agencies/companies
- Discussion

In [18]:
Markdown(result)

# When Will Men Get to Mars? Exploring the Latest Trends and Predictions

## Introduction
Since the dawn of space exploration, humans have looked towards the stars with a sense of wonder and curiosity. The idea of reaching Mars, the next frontier in space exploration, has captured the imagination of scientists, engineers, and dreamers alike. With key players such as NASA, SpaceX, and other space agencies and companies leading the charge, the question on everyone's mind is: when will men finally set foot on the red planet? In this blog post, we will delve into the latest trends and predictions surrounding human missions to Mars, exploring the possibilities and challenges that lie ahead.

## Latest Trends in Mars Exploration
Recent missions to Mars, such as NASA's Perseverance rover and SpaceX's Starship prototypes, have provided us with valuable insights into the Martian landscape and potential habitability. These missions have paved the way for future human exploration by testing new technologies and conducting crucial research. With advancements in propulsion systems, life support technologies, and sustainability solutions, we are closer than ever to making the journey to Mars a reality. The collaboration between public and private entities in space exploration has accelerated progress and opened up new possibilities for reaching the red planet.

## Predictions on When Men Will Get to Mars
Experts in the space industry have put forth various predictions on when humans will finally reach Mars. NASA's Artemis program aims to return humans to the moon by 2024 as a stepping stone towards Mars, with tentative timelines for a Mars mission in the 2030s. SpaceX's ambitious plans envision crewed missions to Mars as early as the mid-2020s, fueled by their reusable rocket technology and Mars colonization efforts. Factors such as funding, technological advancements, and international cooperation will play a crucial role in determining the timeline for human missions to Mars. As we push the boundaries of space exploration, the dream of setting foot on Mars is becoming increasingly tangible.

## Target Audience Analysis
For space enthusiasts, science fiction fans, students, and individuals curious about the future of space exploration, the journey to Mars represents a thrilling adventure into the unknown. The prospect of colonizing Mars and expanding human civilization beyond Earth ignites a sense of wonder and excitement. However, impatience for progress and concerns about the feasibility of Mars missions are common pain points among the audience. By addressing these concerns and sharing the latest updates on Mars exploration, we can inspire a new generation of space explorers and foster a deeper understanding of the challenges ahead.

## Call to Action
As we continue to follow the developments in Mars exploration, it is essential to stay informed and engaged with reputable sources such as NASA and SpaceX. By supporting ongoing missions and participating in discussions about the future of space exploration, we can contribute to the advancement of human civilization beyond Earth. For further reading on Mars exploration and space travel, resources such as NASA's official website, SpaceX's Mars plans, and Space.com offer valuable insights and updates on the latest developments in the field.

In conclusion, the journey to Mars is not just a scientific endeavor but a testament to human ingenuity and perseverance. With the latest trends and predictions pointing towards an imminent reality of human missions to Mars, the dream of exploring the red planet is within reach. By embracing the challenges and opportunities that lie ahead, we can pave the way for a future where men will finally set foot on Mars, marking a new chapter in the history of space exploration.

 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).